In [1]:
%load_ext autoreload
%aimport os, pandas, numpy, pickle
%autoreload 2

In [129]:
from gym_torcs import TorcsEnv
from fqi.reward_function import *
from fqi.sars_creator import to_SARS
from fqi.utils import *
import pandas as pd
import numpy as np
from stable_baselines.ddpg.policies import MlpPolicy
from stable_baselines.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from ddpg.ddpg_driver import DDPG
from stable_baselines.common.math_util import scale_action
from stable_baselines.common.schedules import LinearSchedule

from stable_baselines.deepq.replay_buffer import PrioritizedReplayBuffer

from ddpg.replay_buffer_demonstrations import PrioritizedReplayBufferDemonstrations

In [131]:
ref_df = pd.read_csv('trajectory/ref_traj.csv')
ref_df.columns = ref_traj_cols

simulations = pd.read_csv('trajectory/dataset_offroad_human.csv')
simulations = simulations[simulations.NLap == 17].head(10)
# Reward function
penalty = LikelihoodPenalty(kernel='gaussian', bandwidth=1.0)
right_laps = np.array([ 1.,  8.,  9., 11., 14., 16., 17., 20., 45., 46., 49.,  59., 62.])
penalty.fit(simulations[simulations.NLap.isin(right_laps)][penalty_cols].values)
reward_function = Temporal_projection(ref_df, penalty=penalty)
dataset = to_SARS(simulations, reward_function)

env = TorcsEnv(reward_function, state_cols=state_cols, ref_df=ref_df, vision=False, throttle=True,
               gear_change=False, brake=True, start_env=False, damage_th=3, slow=False, graphic=True)

## inizialize PRBD with demonstrations

In [132]:
#PRB = PrioritizedReplayBuffer(16, 0.005)
demonstrations = list(zip(dataset[state_cols].values,
                         dataset[action_cols].values,
                         dataset['r'].values,
                         dataset[state_prime_cols].values,
                         dataset['absorbing'].values))
"""for t in batch_samples:
    scaled_a = scale_action(env.action_space, t[1])
    PRBD.add(t[0], scaled_a, *t[2:])"""
PRBD = PrioritizedReplayBufferDemonstrations(4, 0.005, demonstrations, env.action_space)


print('Fill Replay Buffer')


PRBD Initialized
Fill Replay Buffer


## Add non-demonstrations transitions

In [139]:
simulations = pd.read_csv('trajectory/dataset_offroad_human.csv')
simulations = simulations[simulations.NLap == 17].tail(3)
dataset = to_SARS(simulations, reward_function)

batch_samples = list(zip(dataset[state_cols].values,
                         dataset[action_cols].values,
                         dataset['r'].values,
                         dataset[state_prime_cols].values,
                         dataset['absorbing'].values))
for t in batch_samples:
    scaled_a = scale_action(env.action_space, t[1])
    PRBD.add(t[0], scaled_a, *t[2:])
    print(PRBD._next_idx)

new _next_idx
12
new _next_idx
9


In [144]:
simulations = pd.read_csv('trajectory/dataset_offroad_human.csv')
simulations[simulations.NLap == 17].head(10)

,NLap,time,isReference,is_partial,xCarWorld,yCarWorld,nYawBody,nEngine,NGear,prevaSteerWheel,...,actualAccelerationX,actualAccelerationY,referenceAccelerationX,referenceAccelerationY,accelerationDiffX,accelerationDiffY,trackPos,aSteerWheel,pBrakeF,rThrottlePedal
11893,17.0,0.055996,1.0,0.0,632.738159,1167.866089,0.019937,18257.00928,7.0,-0.001475,...,1.355255,-3.966545,1.173112,-8.450459,-0.182143,-4.483914,-0.290883,-0.006909,0.0,1.0
11894,17.0,0.155996,1.0,0.0,641.394775,1168.036987,0.016761,18284.10156,7.0,-0.006909,...,1.315348,-2.214881,1.253555,-0.681178,-0.061793,1.533703,-0.259810,-0.002224,0.0,1.0
11895,17.0,0.255996,1.0,0.0,650.064575,1168.186279,0.016794,18309.77905,7.0,-0.002224,...,1.315140,-0.185807,1.173112,-8.450459,-0.142028,-8.264652,-0.232666,-0.000000,0.0,1.0
11896,17.0,0.355996,1.0,0.0,658.747070,1168.332764,0.016888,18335.33691,7.0,-0.000000,...,1.291191,0.008633,1.173112,-8.450459,-0.118079,-8.459092,-0.206033,-0.000000,0.0,1.0
11897,17.0,0.455996,1.0,0.0,667.441956,1168.479248,0.016886,18360.43213,7.0,-0.000000,...,1.266658,0.000317,1.173112,-8.450459,-0.093546,-8.450776,-0.179399,-0.000000,0.0,1.0
11898,17.0,0.555996,1.0,0.0,676.149048,1168.625732,0.016886,18385.04517,7.0,-0.000000,...,1.248418,-0.000128,1.173112,-8.450459,-0.075306,-8.450331,-0.152765,-0.000000,0.0,1.0
11899,17.0,0.655996,1.0,0.0,684.868042,1168.772217,0.016886,18409.16504,7.0,-0.000000,...,1.217150,-0.000074,1.173112,-8.450459,-0.044038,-8.450385,-0.126132,-0.000000,0.0,1.0
11900,17.0,0.755996,1.0,0.0,693.598755,1168.918701,0.016886,18432.93701,7.0,-0.000000,...,1.190268,-0.000060,1.173112,-8.450459,-0.017156,-8.450399,-0.099498,-0.000000,0.0,1.0
11901,17.0,0.855996,1.0,0.0,702.340820,1169.065186,0.016886,18456.39771,7.0,-0.000000,...,1.160735,-0.000112,1.173112,-8.450459,0.012377,-8.450347,-0.072865,-0.000000,0.0,1.0
11902,17.0,0.955996,1.0,0.0,711.094299,1169.211670,0.016886,18478.44971,7.0,-0.000000,...,1.174851,0.000094,1.173112,-8.450459,-0.001739,-8.450553,-0.046231,-0.000000,0.0,1.0


In [141]:
simulations[simulations.NLap == 17].tail(3)

,NLap,time,isReference,is_partial,xCarWorld,yCarWorld,nYawBody,nEngine,NGear,prevaSteerWheel,...,actualAccelerationX,actualAccelerationY,referenceAccelerationX,referenceAccelerationY,accelerationDiffX,accelerationDiffY,trackPos,aSteerWheel,pBrakeF,rThrottlePedal
12610,17.0,71.755996,1.0,0.0,606.796204,1169.167236,0.014648,18377.27417,7.0,-0.005638,...,1.228092,-4.834238,1.173112,-8.450459,-0.054980,-3.616221,0.027233,-0.012017,0.0,1.0
12611,17.0,71.855996,1.0,0.0,615.511414,1169.280029,0.005289,18401.65405,7.0,-0.012017,...,1.204400,-8.053447,1.173112,-8.450459,-0.031288,-0.397012,0.004339,-0.015695,0.0,1.0
12612,17.0,71.955996,1.0,0.0,624.238464,1169.314819,-0.004706,18424.90479,7.0,-0.015695,...,1.180808,-8.502037,1.173112,-8.450459,-0.007696,0.051578,-0.017134,-0.015314,0.0,1.0


In [142]:
PRBD._storage

[(array([ 6.32738159e+02,  1.16786609e+03,  1.99370000e-02,  1.82570093e+04,
          6.05376587e+00, -2.36307390e-01, -3.14047628e+00, -3.12883623e+00,
          3.11462320e+02,  3.08826794e+00,  1.35525500e+00, -3.96654500e+00,
          1.17311200e+00, -8.45045900e+00, -1.82143000e-01, -4.48391400e+00,
         -1.47500000e-03,  0.00000000e+00,  1.00000000e+00]),
  array([-0.006909, -1.      ,  1.      ]),
  0.0,
  array([ 6.41394775e+02,  1.16803699e+03,  1.67610000e-02,  1.82841016e+04,
          2.73074634e+02, -1.57213008e+00, -3.14159151e+00, -3.13941119e+00,
          3.11925343e+02,  1.64380911e+00,  1.31534800e+00, -2.21488100e+00,
          1.25355500e+00, -6.81178000e-01, -6.17930000e-02,  1.53370300e+00,
         -6.90900000e-03,  0.00000000e+00,  1.00000000e+00]),
  False),
 (array([ 6.41394775e+02,  1.16803699e+03,  1.67610000e-02,  1.82841016e+04,
          2.73074634e+02, -1.57213008e+00, -3.14159151e+00, -3.13941119e+00,
          3.11925343e+02,  1.64380911e+00,  1

In [178]:
batch_size = 100
total_timesteps = 1000
prioritized_replay_beta_iters = total_timesteps
prioritized_replay_beta0 = 0.4
num_timesteps = 1
beta_schedule = LinearSchedule(prioritized_replay_beta_iters,
                                                    initial_p=prioritized_replay_beta0,
                                                    final_p=1.0)
experience = PRBD.sample(batch_size, beta=beta_schedule.value(num_timesteps))
(obs, actions, rewards, next_obs, dones, weights, batch_idxes) = experience
experience

(array([[ 6.41394775e+02,  1.16803699e+03,  1.67610000e-02, ...,
         -6.90900000e-03,  0.00000000e+00,  1.00000000e+00],
        [ 7.02340820e+02,  1.16906519e+03,  1.68860000e-02, ...,
         -0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
        [ 6.93598755e+02,  1.16891870e+03,  1.68860000e-02, ...,
         -0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
        ...,
        [ 6.58747070e+02,  1.16833276e+03,  1.68880000e-02, ...,
         -0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
        [ 6.67441956e+02,  1.16847925e+03,  1.68860000e-02, ...,
         -0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
        [ 6.84868042e+02,  1.16877222e+03,  1.68860000e-02, ...,
         -0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
 array([[-0.002224, -1.      ,  1.      ],
        [ 0.      , -1.      ,  1.      ],
        [ 0.      , -1.      ,  1.      ],
        [-0.012017, -1.      ,  1.      ],
        [ 0.      , -1.      ,  1.      ],
        [ 0.  

In [124]:
len(PRBD._storage)

9

In [104]:
PRB = PrioritizedReplayBuffer(8, 0.005)

In [113]:
simulations = pd.read_csv('trajectory/dataset_offroad_human.csv')
simulations = simulations[simulations.NLap == 17].tail(10)
dataset = to_SARS(simulations, reward_function)

batch_samples = list(zip(dataset[state_cols].values,
                         dataset[action_cols].values,
                         dataset['r'].values,
                         dataset[state_prime_cols].values,
                         dataset['absorbing'].values))
for t in batch_samples:
    scaled_a = scale_action(env.action_space, t[1])
    PRB.add(t[0], scaled_a, *t[2:])

In [114]:
len(PRB._storage)

8